In [4]:
### Vaccination Index Predictions

In [2]:
# Import your dependencies
# Initial imports
import pandas as pd
from path import Path
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn import *
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# Read in the file
data = Path('../Resources/merged_dataset.csv')
county_df = pd.read_csv(data)
county_df.head()


### Data Preprocessing for Machine Learning 

### Label Encoder - Replaces non numerical data with numbers 
### Can only use numerical values (Update City, County, and State)

In [8]:
# County, State, and City
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
county_df = county_df.copy()
# county_df['le_county'] = le.fit_transform(county_df['County'])
# county_df['le_state'] = le.fit_transform(county_df['State'])
# county_df['le_city'] = le.fit_transform(county_df['City'])
county_df2 = county_df.drop(columns = ['County', 'Latitude', 'Longitude','State','City'])
county_df2

,FIPS,UI_Index,MI_Index,CI_Index,Total_Vaccinated,Vac_Index
0,1001,4.9,112.5,90.9,2288180,46.7
1,1001,4.9,112.5,85.7,2288180,46.7
2,1001,4.9,112.5,91.9,2288180,46.7
3,1001,4.9,112.5,90.7,2288180,46.7
4,1001,4.9,112.5,90.5,2288180,46.7
...,...,...,...,...,...,...
41803,56041,6.3,107.0,96.9,268472,46.4
41804,56043,5.3,83.3,94.2,268472,46.4
41805,56043,5.3,83.3,96.9,268472,46.4
41806,56045,3.9,89.8,94.2,268472,46.4


In [9]:
### Update numerical values to Boolean

In [10]:
# Convert 'Median_Income_Index' column to Boolean
indexes_df= county_df2['MI_Index']>=100
# Add the converted column back to the DataFrame
county_df2['MI_Index'] = indexes_df.astype(int)
county_df2


,FIPS,UI_Index,MI_Index,CI_Index,Total_Vaccinated,Vac_Index
0,1001,4.9,1,90.9,2288180,46.7
1,1001,4.9,1,85.7,2288180,46.7
2,1001,4.9,1,91.9,2288180,46.7
3,1001,4.9,1,90.7,2288180,46.7
4,1001,4.9,1,90.5,2288180,46.7
...,...,...,...,...,...,...
41803,56041,6.3,1,96.9,268472,46.4
41804,56043,5.3,0,94.2,268472,46.4
41805,56043,5.3,0,96.9,268472,46.4
41806,56045,3.9,0,94.2,268472,46.4


In [11]:
indexes_df= county_df2['CI_Index']>=100
county_df2['CI_Index'] = indexes_df.astype(int)
county_df2

,FIPS,UI_Index,MI_Index,CI_Index,Total_Vaccinated,Vac_Index
0,1001,4.9,1,0,2288180,46.7
1,1001,4.9,1,0,2288180,46.7
2,1001,4.9,1,0,2288180,46.7
3,1001,4.9,1,0,2288180,46.7
4,1001,4.9,1,0,2288180,46.7
...,...,...,...,...,...,...
41803,56041,6.3,1,0,268472,46.4
41804,56043,5.3,0,0,268472,46.4
41805,56043,5.3,0,0,268472,46.4
41806,56045,3.9,0,0,268472,46.4


In [12]:
indexes_df= county_df2['UI_Index']<=6
county_df2['UI_Index'] = indexes_df.astype(int)
county_df2

,FIPS,UI_Index,MI_Index,CI_Index,Total_Vaccinated,Vac_Index
0,1001,1,1,0,2288180,46.7
1,1001,1,1,0,2288180,46.7
2,1001,1,1,0,2288180,46.7
3,1001,1,1,0,2288180,46.7
4,1001,1,1,0,2288180,46.7
...,...,...,...,...,...,...
41803,56041,0,1,0,268472,46.4
41804,56043,1,0,0,268472,46.4
41805,56043,1,0,0,268472,46.4
41806,56045,1,0,0,268472,46.4


In [13]:
indexes_df= county_df2['Vac_Index']>=60
county_df2['Vac_Index'] = indexes_df.astype(int)
county_df2

,FIPS,UI_Index,MI_Index,CI_Index,Total_Vaccinated,Vac_Index
0,1001,1,1,0,2288180,0
1,1001,1,1,0,2288180,0
2,1001,1,1,0,2288180,0
3,1001,1,1,0,2288180,0
4,1001,1,1,0,2288180,0
...,...,...,...,...,...,...
41803,56041,0,1,0,268472,0
41804,56043,1,0,0,268472,0
41805,56043,1,0,0,268472,0
41806,56045,1,0,0,268472,0


In [14]:
# EXPORT TO CSV FOR TABLEAU AND FUTURE MODELS
county_df2.to_csv('encoded_dataset.csv', index=False)

In [26]:
county_file = pd.read_csv('~/Desktop/Bootcamp Folder/FinalProject/Resources/Datasets/Tableau Data/encoded_dataset.csv')

In [27]:
county_df_to_db = pd.DataFrame(county_file)
county_df_to_db

,FIPS,UI_Index,MI_Index,CI_Index,Total_Vaccinated,Vac_Index
0,1001,1,1,0,2288180,0
1,1001,1,1,0,2288180,0
2,1001,1,1,0,2288180,0
3,1001,1,1,0,2288180,0
4,1001,1,1,0,2288180,0
...,...,...,...,...,...,...
41803,56041,0,1,0,268472,0
41804,56043,1,0,0,268472,0
41805,56043,1,0,0,268472,0
41806,56045,1,0,0,268472,0


In [29]:
school_file = pd.read_csv('~/Downloads/school_job_data.csv')

In [30]:
school_df_to_db = pd.DataFrame(school_file)
school_df_to_db

,School,City_x,State,County,Latitude,Longitude,JobTitle,Company,City_y
0,University of Arkansas at Little Rock,Little Rock,AR,Pulaski,34.723254,-92.339778,Senior Data Analyst,Walmart,Fayetteville
1,University of Arkansas at Little Rock,Little Rock,AR,Pulaski,34.723254,-92.339778,Senior Data Analyst,Walmart,Fayetteville
2,University of Arkansas at Little Rock,Little Rock,AR,Pulaski,34.723254,-92.339778,Senior Data Analyst,Walmart,Fayetteville
3,University of Arkansas at Little Rock,Little Rock,AR,Pulaski,34.723254,-92.339778,Senior Data Analyst,Walmart,Fayetteville
4,University of Arkansas at Little Rock,Little Rock,AR,Pulaski,34.723254,-92.339778,Senior Data Analyst,Walmart,Fayetteville
...,...,...,...,...,...,...,...,...,...
1971979,Johnny Matthew's Hairdressing Training School,Salem,OR,Marion,44.941812,-123.038361,Ubuntu on WSL Software Engineer,Canonical,Portland
1971980,Johnny Matthew's Hairdressing Training School,Salem,OR,Marion,44.941812,-123.038361,Software Engineer II – Identity (Open to remot...,Nike,Portland
1971981,Johnny Matthew's Hairdressing Training School,Salem,OR,Marion,44.941812,-123.038361,Software Engineer Intern - Financial Analytics...,Addepar,Portland
1971982,Johnny Matthew's Hairdressing Training School,Salem,OR,Marion,44.941812,-123.038361,Business Systems Analyst,Dice,Portland


In [28]:
engine = create_engine(f"postgresql://postgres:<PASSWORD>@finalproject.cmaigqkcfkxs.us-east-2.rds.amazonaws.com:5432/FinalProject")
county_df_to_db.to_sql(name='encoded_dataset', con=engine, index=False, if_exists="replace")

In [32]:
engine = create_engine(f"postgresql://postgres:<PASSWORD>@finalproject.cmaigqkcfkxs.us-east-2.rds.amazonaws.com:5432/FinalProject")
school_df_to_db.to_sql(name='school_job_data', con=engine, index=False, if_exists="replace")